In [1]:
import momi, pandas, os

Use the **`help()`** function to view documentation.

In [2]:
help(momi)

Help on package momi:

NAME
    momi

FILE
    /Users/jkamm/anaconda/lib/python2.7/site-packages/momi/__init__.py

DESCRIPTION
    momi (MOran Models for Inference) is a python package for computing the site frequency spectrum,
    a summary statistic commonly used in population genetics, and using it to infer demographic history.
    
    Please refer to examples/tutorial.ipynb for usage & introduction.

PACKAGE CONTENTS
    compute_sfs
    convolution
    data_structure
    demography
    likelihood
    math_functions
    moran_model
    parse_ms
    size_history
    tensor
    util




# Creating demographies

momi uses a syntax based on the program ms by Richard Hudson.
A demography is specified as a sequence of events.
Time is measured going backwards from the present (t=0) to the past (t>0).

There are 4 kinds of events:
* **('-en', t, i, N)**
    * At time t, population i has its scaled population size set to N, and growth rate set to 0.
* **('-eg', t, i, g)**
    * At time t, population i has exponential growth rate g (so for s>t, $N(s) = N(t) e^{(s-t)  g}$)
* **('-ej', t, i, j)**
    * At time t, all lineages in population i move into j.
* **('-ep', t, i, j, p_ij)**
    * At time t, each lineage in i moves into j with probability p_ij.

Note **-en,-eg,-ej** are flags from ms, while **-ep** replaces the flag **-es** in ms.
By default, all parameters are scaled as in ms, but this can be adjusted.

See **`help(momi.Demography)`** or **`help(momi.Demography.__init__)`** for more details.

### An example demography

Here we consider a concrete example. More examples can be found at [example_demographies.ipynb](files/example_demographies.ipynb).

Unlike ms, populations can be labeled by arbitrary strings.  
Here we label the sampled populations as **'chb'** and **'yri'**.  
The demography also involves admixture with a third population, **'nea'**.

By default, the parameters are scaled like in **ms**. The population sizes are assumed to be rescaled by a "reference" size N_ref (e.g. 10,000), and time is scaled so there are 4*N_ref generations per unit time.

In [3]:
# define the list of events
events = [('-en', 0., 'chb', 10.),         # at present (t=0), 'chb' has diploid population size 10 * N_ref
          ('-eg', 0, 'chb' , 6.),          # at present (t=0), 'chb' growing at rate 6
          ('-ep', .25, 'chb', 'nea', .03), # at t=.25, 'chb' has a bit of admixture from 'nea'
          ('-ej', .5, 'chb', 'yri'),       # at t=.5, 'chb' joins onto 'yri' 
          ('-ej', 1.5, 'yri', 'nea'),      # at t=1.5, 'yri' joins onto 'nea'
          ]

# construct the Demography object, sampling 14 alleles from 'yri' and 10 alleles from 'chb'
demo = momi.make_demography(events, sampled_pops=('yri','chb'), sampled_n=(14,10))

# Coalescent statistics

`momi` can compute coalescent statistics such as the TMRCA (time to most recent common ancestor) and total branch length of the genealogy.

In [4]:
eTmrca = momi.expected_tmrca(demo)
print "Expected TMRCA of all samples:", "\t", eTmrca

eTmrca_chb = momi.expected_deme_tmrca(demo, 'chb')
print "Expected TMRCA of chb samples:", "\t", eTmrca_chb

eL = momi.expected_total_branch_len(demo)
print "Expected total branch length:", "\t", eL

# See help(momi.expected_tmrca), etc. for more details.
# Advanced users can use momi.expected_sfs_tensor_prod()
# to compute these and many other summary statistics.

Expected TMRCA of all samples: 	1.41920517653
Expected TMRCA of chb samples: 	1.25374295456
Expected total branch length: 	7.93963743415


# Expected Sample Frequency Spectrum (SFS)

The expected SFS for configuration $((a_0,d_0),(a_1,d_1),...)$ is the expected number of SNPs with $a_0$ ancestral and $d_0$ derived alleles in population 0, $a_1$ ancestral and $d_1$ derived alleles in population 1, etc.

The following all create a configuration with 13 ancestral, 1 derived alleles in "yri", and 10 ancestral, 0 derived alleles in "chb".

In [5]:
# these all represent the same config
[[13,1],[10,0]]
momi.config(a=(13,10), d=(1,0))
momi.config(d=(1,0), n=(14,10))
momi.config(a=(13,10), n=(14,10))

((13, 1), (10, 0))

Use **`momi.Configs`** to represent a list of configs, and **`momi.expected_sfs()`** to compute their expected SFS.

In [6]:
# a list of configs (index0 == yri, index1 == chb)
configs = [( (14,0), (9,1), ), # 1 derived allele in chb
           ( (13,1), (10,0), ), # 1 derived allele in yri 
           ( (11,3), (9,1),) , # 3 derived in yri, 1 derived in chb
           ( (14,0), (0,10), ), # 0 derived in yri, all derived in chb
           ( (2,12), (10,0), ), # 12 derived in yri, 0 derived in chb 
           ( (2,12), (2,8), ), # 12 derived in yri, 8 derived in chb
          ]

configs = momi.Configs(("yri","chb"), configs)

# the SFS entries corresponding to each config in configs
eSFS = momi.expected_sfs(demo, configs, mut_rate=1.0)
print eSFS

# See help(momi.expected_sfs) for more options (e.g. folded SFS, sampling error, normalization)

[ 2.6309565   0.95647102  0.0048574   0.06108049  0.04371188  0.00323204]


# Segregating sites

A dataset of segregating sites has been stored in [tutorial_data.txt](tutorial_data.txt). The file is organized as follows:
* Each locus starts with a line "**//**".
* Subsequent lines correspond to segregating sites.
    * The first column is the **position** of the site $x \in [0,1]$.
    * Subsequent columns give the **ancestral,derived** allele counts (respectively), in each population.

In [7]:
# print first few lines in the shell
!head tutorial_data.txt

Position	:	yri	chb

//

0.0	:	14,0	7,3
0.0007	:	13,1	10,0
0.001	:	0,14	10,0
0.0012	:	11,3	10,0
0.0012	:	14,0	9,1
0.0013	:	0,14	10,0


**`momi.read_seg_sites`** reads the file, and returns a **`momi.SegSites`** object.

In [8]:
# read the file with momi
data_filename = "tutorial_data.txt"
with open(data_filename,'r') as f:
    seg_sites = momi.read_seg_sites(f)

**`momi.SegSites`** has methods **`SegSites.position()`**, **`SegSites.config()`**, and **`SegSites.allele_count()`** to access the positions and allele counts at different sites.

**`SegSites.position_arrays`** and **`SegSites.config_arrays`** are read-only arrays of the positions and configs.

In [9]:
print seg_sites.position(locus=0,site=2) # equivalent to seg_sites.position_arrays[0][2]
print seg_sites.config(locus=0, site=2) # equivalent to seg_sites.config_arrays[0][2,:,:]
print seg_sites.allele_count(locus=0, site=2, pop=1, allele=0) # equivalent to seg_sites.config_arrays[0][2,1,0]

0.001
[[ 0 14]
 [10  0]]
10


**`momi.SegSites`** can be directly constructed with the arrays of configs and positions.

In [10]:
seg_sites2 = momi.SegSites(seg_sites.sampled_pops, 
                           seg_sites.config_arrays, 
                           seg_sites.position_arrays)

assert seg_sites2 == seg_sites

# Observed SFS 

**`momi.Sfs`** represents the observed SFS, i.e. the empirical frequencies of each config.  

In [11]:
# the Sfs for the observed data
sfs = seg_sites.sfs

**`Sfs.freq()`** returns the per-locus or total frequency of a config.    
**`Sfs.loci`** and **`Sfs.total`** are read-only dicts of the per-locus and total frequencies, respectively.

In [12]:
print sfs.freq(  ((13,1) , (10,0))  ) # equivalent to sfs.total[((13,1),(10,0))]
print sfs.freq( ((13,1) , (10,0)) , locus=0) # equivalent to sfs.loci[0][((13,1),(10,0))]

print "Unique configs:", len(sfs.total)

9251
412
Unique configs: 163


**`momi.Sfs`** can be directly constructed from the per-locus frequencies.

In [13]:
# construct using the per-locus frequencies
sfs2 = momi.Sfs(sfs.sampled_pops, sfs.loci)

assert sfs2 == sfs

# Composite likelihood

The composite likelihood treats each site as independent.  
`momi` provides 2 composite likelihoods:
* **multinomial**: Fixed number of sites, each drawn from multinomial
* **Poisson random field**: Random number of sites, drawn from Poisson distribution

Here we illustrate the **multinomial** likelihood (the default). See **help(momi.composite_log_likelihood)** for more options.

In [14]:
print "Composite log likelihood:", momi.SfsLikelihoodSurface(sfs).log_likelihood(demo)
# momi.composite_log_likelihood(seg_sites, demo) also works

Composite log likelihood: -208068.033149


# Automatic differentiation

`momi` uses the package **`autograd`** to automatically compute derivatives.

Gradients can be extremely useful in parameter inference.
However, computing gradients is *not* necessary for basic functionality.
Users who don't plan to use auto-differentiation can skip this section, or return to it later.

To start, define a function that maps some parameters to a demography.

In [15]:
import autograd
import autograd.numpy as np ## thinly wrapped version of numpy for auto-differentiation

def demo_func(N_chb_bottom, N_chb_top, pulse_t, pulse_p, ej_chb, ej_yri):
    ej_chb = pulse_t + ej_chb
    ej_yri = ej_chb + ej_yri
    
    # use autograd.numpy for math functions (e.g. logarithm)
    # This will allow us to take derivatives later
    G_chb = -np.log(N_chb_top / N_chb_bottom) / ej_chb
    
    events = [('-en', 0., 'chb', N_chb_bottom),
              ('-eg', 0, 'chb' , G_chb),
              ('-ep', pulse_t, 'chb', 'nea', pulse_p),
              ('-ej', ej_chb, 'chb', 'yri'),
              ('-ej', ej_yri, 'yri', 'nea'),
              ]

    return momi.make_demography(events, ('yri','chb'), (14,10))

Use **`autograd`** to compute arbitrary gradients.  
For example, here we compute the gradient of the TMRCA.

In [16]:
# function mapping vector of parameters to the TMRCA of the corresponding demography
def tmrca_func(params):
    # equivalent to demo_func(params[0], params[1], params[2], ...)
    demo = demo_func(*params)
    # return expected TMRCA
    return momi.expected_tmrca(demo)

# use autograd.grad() to obtain the gradient function
tmrca_grad = autograd.grad(tmrca_func)

x = [10., .1, .25, .03, .25, 1.]
print "Parameters:", x
print "Expected TMRCA:", tmrca_func(x)
print "Gradient:"
print tmrca_grad(x)

Parameters: [10.0, 0.1, 0.25, 0.03, 0.25, 1.0]
Expected TMRCA: 1.31277716218
Gradient:
[0.0045248167517458011, 0.55838496083537004, 0.28449869706692488, 3.9671067152725392, 0.83175299796269697, 0.13916390098743361]


### More details about using `autograd` for automatic differentiation

`autograd` uses the magic of *operator overloading* to compute derivatives automatically.

Documentation for autograd can be found at https://github.com/HIPS/autograd/

Some do's and don'ts for autograd (from the tutorial):
* Do use
    * Arithmetic operations `+,-,*,/,**`
    * Most of `numpy`'s functions
    * Most `numpy.ndarray` methods
    * Some `scipy` functions
    * Indexing and slicing of arrays `x = A[3, :, 2:4]`
    * Explicit array creation from lists `A = np.array([x, y])`
* Don't use
    * Assignment to arrays `A[0,0] = x`
    * Implicit casting of lists to arrays `A = numpy.sum([x, y])`, use `A = numpy.sum(np.array([x, y]))` instead.
    * `A.dot(B)` notation (use `numpy.dot(A, B)` instead)
    * In-place operations (such as `a += b`, use `a = a + b` instead)

# Inference

We'll try to infer the following demography from data.

In [17]:
true_params = [10., .1, .25, .03, .25, 1.]
true_demo = demo_func(*true_params)

We can generate a new dataset with **`ms`** (or similar program, e.g. `scrm`, `macs`, `msprime`, etc.)

In [18]:
## to generate new dataset, change this to
## ms_path = "/path/to/ms"
ms_path = None
#ms_path = os.environ["MS_PATH"]

if ms_path is not None:
    print "Generating new dataset with ms"
    
    n_loci = 20
    kb_per_locus = 500
    mut_rate_per_kb, recom_rate_per_kb = 1.,1.
    
    seg_sites = momi.simulate_ms(ms_path, true_demo, 
                                 n_loci, kb_per_locus * mut_rate_per_kb, 
                                 additional_ms_params="-r %f %d" % (kb_per_locus*recom_rate_per_kb,
                                                                    int(kb_per_locus * 1000)))
    sfs = seg_sites.sfs
    
    ## uncomment these lines to save the new dataset
    #with open(data_filename,'w') as f:
    #    momi.write_seg_sites(f, seg_sites)
else:
    print "No ms_path provided, using SFS previously stored in %s." % data_filename

No ms_path provided, using SFS previously stored in tutorial_data.txt.


In [19]:
# define (lower,upper) bounds on the parameter space
bounds = [(.01, 100.),
          (.01, 100.),
          (.01,5.),
          (1e-100,.25),
          (.01,5.),
          (.01,5.)]

# pick a random start value for the parameter search
import random
lower_bounds, upper_bounds = [l for l,u in bounds], [u for l,u in bounds]
start_params = [random.triangular(lower, upper, mode)
                for lower, upper, mode in zip(lower_bounds, upper_bounds, [1, 1, 1, .1, 1,1,1])]

Search for the MCLE (max composite likelihood estimate) with **`momi.composite_mle_search()`**.

By default, `momi.composite_mle_search()` assumes `demo_func` is differentiable with `autograd`, and uses the gradient in a hill-climbing algorithm.
* If you don't want to use `autograd`, you can disable the gradient (i.e. jacobian) with:
    * `momi.composite_mle_search(..., jac=False, ...)`
* Conversely, `momi.composite_mle_search()` provides options for using the Hessian (second-order derivative), in addition to the gradient.
    * See **`help(momi.composite_mle_search)`**.

Beware that **`momi.composite_mle_search()`** is vulnerable to local maxima.  

In [20]:
surface = momi.SfsLikelihoodSurface(seg_sites, demo_func)
mcle_search_result = surface.find_optimum(start_params, bounds = bounds, maxiter = 500, output_progress = 5)

iter 0: KL-Divergence([  4.87614892e+01   2.19212946e+01   1.00000000e-02   9.25896395e-02
   1.97887257e+00   1.85132494e+00]) == 0.729572
iter 5: KL-Divergence([  2.63212356e+01   9.86508524e-02   7.35843904e-02   7.30677964e-03
   4.43969217e-01   7.90608252e-01]) == 0.018362
iter 10: KL-Divergence([ 9.30454172  0.11378546  0.22338447  0.02360587  0.3004668   0.98211669]) == 0.003010
iter 15: KL-Divergence([ 10.47089542   0.09823204   0.25820117   0.03495311   0.25522688
   0.92599826]) == 0.002648


In [21]:
# print info such as whether search succeeded, number function/gradient evaluations, etc
print "Search results:"
print mcle_search_result

Search results:
  status: 1
 success: True
    nfev: 90
     fun: 0.0026476115430336738
       x: array([ 10.43689601,   0.09823029,   0.25827779,   0.03520851,
         0.25496233,   0.91948918])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
     jac: array([  4.37590996e-08,  -1.61167821e-06,   3.23757777e-05,
         2.26078440e-05,  -8.05900525e-07,   8.52466794e-07])
     nit: 18


In [22]:
est_params = mcle_search_result.x
print "Estimated params:"
print est_params

Estimated params:
[ 10.43689601   0.09823029   0.25827779   0.03520851   0.25496233
   0.91948918]


# Confidence regions and Hypothesis testing

`momi` uses the **Limit of Experiments** to compute approximate confidence regions and hypothesis tests.  
A good reference is *Asymptotic Statistics* by **A.W. van der Vaart**.  
(Some readers may be familiar with the **Godambe Information**, which uses a special case of this theory.)

If certain regularity conditions hold, then the confidence regions will have the correct coverage
properties as the data $\to \infty$.  
In practice it is important to check the coverage via simulation, because:
* It is not well understood when the regularity conditions hold in demographic inference.
* Even if the regularity conditions hold, the coverage will be inaccurate if there is not enough data.

`momi` can construct the asymptotic confidence regions for 2 limiting regimes:
* **"long"**: fixed number of loci, whose length $\to \infty$
    * Treats the segregating sites along each locus as a time series
    * The loci should be independent (unlinked), but don't have to be identically distributed
* **"many"**: many short loci, with the number of loci ` $\to \infty$
    * The loci should be independent, and roughly identically distributed.

In [23]:
# a confidence region around the estimated parameters
confidence_region = momi.ConfidenceRegion(est_params, demo_func, seg_sites, regime="long")

The p-value at the truth, using likelihood ratio test.  
Asymptotically, the log-likelihood ratio is a transformation of a Gaussian.  
We use **`ConfidenceRegion.test()`** to compute an approximate p-value, by an inexpensive simulation of 1000 Gaussians.

In [24]:
print "p-value at true params:", confidence_region.test(true_params, sims=int(1e3))

p-value at true params: 0.628


**`ConfidenceRegion.test()`** can also compute the p-values at a list of points.

In [25]:
confidence_region.test([est_params * .95, est_params * .98, est_params * 1.05], sims=int(1e3))

array([ 0.066,  0.898,  0.088])

**`ConfidenceRegion.test()`** can also construct tests and intervals based on the Wald-statistic.  
This does not require simulation, but typically the Wald statistic is less powerful and converges more slowly than the likelihood ratio.  

In [26]:
print "p-value at true params (Wald):", confidence_region.test(true_params, test_type="wald")

p-value at true params (Wald): 0.511276915808


One advantage of Wald is the ease of computing marginal intervals.

In [27]:
print "95% marginal confidence intervals"
print confidence_region.wald_intervals(lower=.025,upper=.975)

95% marginal confidence intervals
[[  9.39257848  11.48121355]
 [  0.08473396   0.11172662]
 [  0.23593408   0.28062149]
 [  0.02298093   0.04743609]
 [  0.22948322   0.28044143]
 [  0.67013838   1.16883998]]


### Complex hypothesis tests

**`ConfidenceRegion.test()`** can also perform more complicated tests for nested models.  
We consider testing that the pulse probability is 0.  
We will also fix the pulse time and "nea" join time; otherwise the demography is not identifiable, which violates a crucial regularity condition.  
(If the pulse probability is 0, then the pulse time, and the "nea" join on time, have no affect on the likelihood).

In [28]:
# Find the MLE in the null model space
start0, bound0 = list(est_params), list(bounds)
start0[2] = bound0[2] = est_params[2] # fix the pulse time
start0[5] = bound0[5] = est_params[5] # fix the 'nea' join time
start0[3] = bound0[3] = 1e-100 # set the pulse probability to 0

constrained_opt_result = surface.find_optimum(start0, bounds = bound0, maxiter = 500, output_progress = 5)

iter 0: KL-Divergence([ 10.43705793   0.16846947   0.25493375]) == 0.019436
iter 5: KL-Divergence([ 9.01982456  0.1832553   0.29066107]) == 0.018633


In [29]:
print "MLE of null model:"
est0 = constrained_opt_result.x
print est0

MLE of null model:
[  9.03910818e+000   1.83272907e-001   2.58277787e-001   1.00000000e-100
   2.90741822e-001   9.19489181e-001]


In [30]:
# Null model:
# parameters 0,1,4 unconstrained ("None")
# parameters 2,3,5 are fixed ("0")
null_cone = (None,None,0,0,None,0)

# Alternative model:
# parameters 0,1,4, unconstrained ("None")
# parameters 2,5 fixed ("0")
# parameter 3 bounded on the left ("1". use "-1" if bounded on the right)
alt_cone = (None,None,0,1,None,0) 

# approximate p value with 10000 simulations
p0 = confidence_region.test(null_point=est0, alt_point=est_params, 
                            null_cone=null_cone, alt_cone=alt_cone,
                            sims=int(1e4))
print "Approximate p-value of no migration:", p0

Approximate p-value of no migration: 0.0


Note that **ConfidenceRegion.test()** returns 0 if the likelihood ratio is more extreme than all 10000 null simulations performed.

Also note that the above test involved **testing after model selection** (since we fixed the pulse time and "nea" join time).  
This may be statistically invalid; the p-values should be checked and adjusted by simulation.